# **Finding the Optimal Layout**

In [1]:
from collections import defaultdict
import pandas as pd
import itertools

In [19]:
def compute_loss(keyboard_layout, corpus):
    key_map = {}
    for i, key in enumerate(keyboard_layout):
        for letter in key:
            key_map[letter] = i

    word_patterns = defaultdict(int)
    for word in corpus:
        pattern = []
        last_key = None
        for letter in word:
            key = key_map[letter]
            if key != last_key:
                pattern.append(key)
                last_key = key
        word_patterns[tuple(pattern)] += 1

    # The loss is the number of words that have the same pattern
    loss = sum(count for count in word_patterns.values() if count > 1)
    return loss

In [23]:
def backtrack(current_layout, remaining_letters, max_keys, best_layout, best_loss, corpus):
    # Check if we have assigned all letters and have at most max_keys keys
    print(current_layout)
    if not remaining_letters and len(current_layout) <= max_keys:
        # Evaluate this layout
        loss = compute_loss(current_layout, corpus)
        if loss < best_loss[0]:
            best_loss[0] = loss
            best_layout[:] = current_layout[:]
        return

    # If already have max_keys and there are still letters left, return
    if len(current_layout) >= max_keys:
        return

    # Try to group remaining letters into the next key, considering all possible splits
    for i in range(1, len(remaining_letters) + 1):
        new_key = remaining_letters[:i]
        new_remaining = remaining_letters[i:]
        backtrack(current_layout + [new_key], new_remaining, max_keys, best_layout, best_loss, corpus)

In [21]:
def find_optimal_keyboard(corpus, max_keys=12):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    best_layout = []
    best_loss = [float('inf')]

    backtrack([], alphabet, max_keys, best_layout, best_loss, corpus)

    return best_layout, best_loss[0]

In [5]:
# Example usage:
corpus = ["HELLO", "WORLD", "KEYBOARD", "CIRCUIT", "EXAMPLE", "PYTHON", "OPTIMIZATION", "TASK"]
optimal_layout, minimal_loss = find_optimal_keyboard(corpus)

print("Optimal Layout:", optimal_layout)
print("Minimal Loss:", minimal_loss)


Optimal Layout: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'LMNOPQRSTUVWXYZ']
Minimal Loss: 0


# **Real Testing**

In [24]:
df_vocab = pd.read_csv('../data/vocab_final.csv')
corpus = df_vocab['word'].tolist()

# Transform the None or NaN values to "null"
for i in range(len(corpus)):
    if type(corpus[i]) != str:
        corpus[i] = "null"

optimal_layout, minimal_loss = find_optimal_keyboard(corpus)

print("Optimal Layout:", optimal_layout)
print("Minimal Loss:", minimal_loss)

[]
['a']
['a', 'b']
['a', 'b', 'c']
['a', 'b', 'c', 'd']
['a', 'b', 'c', 'd', 'e']
['a', 'b', 'c', 'd', 'e', 'f']
['a', 'b', 'c', 'd', 'e', 'f', 'g']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lm']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmn']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmno']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmnop']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmnopq']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmnopqr']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmnopqrs']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmnopqrst']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'lmnopqrstu']
[

KeyboardInterrupt: 

In [2]:
def compute_loss(keyboard_layout, corpus):
    key_map = {}
    for i, key in enumerate(keyboard_layout):
        for letter in key:
            key_map[letter] = i

    word_patterns = defaultdict(int)
    for word in corpus:
        pattern = []
        last_key = None
        for letter in word:
            key = key_map[letter]
            if key != last_key:
                pattern.append(key)
                last_key = key
        word_patterns[tuple(pattern)] += 1

    # The loss is the number of words that have the same pattern
    loss = sum(count for count in word_patterns.values() if count > 1)
    return loss

def split_alphabet(alphabet, max_keys):
    n = len(alphabet)
    for indices in itertools.combinations(range(1, n), max_keys - 1):
        yield [alphabet[i:j] for i, j in zip((0,) + indices, indices + (None,))]

def find_optimal_keyboard(corpus, max_keys=12):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    best_layout = []
    best_loss = float('inf')

    for split in split_alphabet(alphabet, max_keys):
        loss = compute_loss(split, corpus)
        if loss < best_loss:
            best_loss = loss
            best_layout = split

    return best_layout, best_loss

# Example usage with a smaller corpus for demonstration
df_vocab = pd.read_csv('../data/vocab_final.csv')
corpus = df_vocab['word'].tolist()

# Transform the None or NaN values to "null"
for i in range(len(corpus)):
    if type(corpus[i]) != str:
        corpus[i] = "null"

optimal_layout, minimal_loss = find_optimal_keyboard(corpus)

print("Optimal Layout:", optimal_layout)
print("Minimal Loss:", minimal_loss)
